In [ ]:
import numpy as np
import pandas as pd
import csv
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, RepeatVector, Dense
from tensorflow.keras.layers import Reshape
from sklearn.preprocessing import MinMaxScaler

##Peaks Normalization and Log Transformation

In [ ]:
data = []
with open('CrestFactor.csv') as f:
    reader = csv.reader(f)
    next(reader) # skip header
    for row in reader:
        float_row = []
        for x in row[1:]:
            if x == '':
                float_row.append(np.nan)
            else:
                float_row.append(float(x))
        data.append(float_row)

# Replace NaN with 0
data = [np.nan_to_num(x) for x in data]

data=pd.DataFrame(data)

data = data.iloc[:61]
timesteps = 61
n_features = data.shape[1]

In [ ]:
data

,0,1,2,3,4,5,6,7,8,9,10
0,3.06397,4.977319,4.191362,3.381538,3.185565,2.354286,3.161872,3.236008,2.639186,3.493800,3.011101
1,3.83101,5.660888,2.443565,2.365860,2.478942,2.668000,3.314006,7.299812,3.011938,6.687019,2.576116
2,4.05691,3.852273,3.297534,2.808909,2.326404,2.572775,3.263217,6.899070,2.537908,3.230957,3.574377
3,3.52221,3.732933,2.776657,2.976186,3.106064,3.172916,3.112790,4.458837,2.764861,3.135373,3.503921
4,3.98526,4.224989,3.141246,2.479858,3.310935,2.447641,2.625999,3.694837,2.240795,3.460449,3.322744
...,...,...,...,...,...,...,...,...,...,...,...
56,2.77059,3.754850,2.905992,2.810311,2.708957,2.755360,2.966177,2.823075,3.094206,3.073613,2.861180
57,4.50932,3.888347,3.045900,2.506961,2.386530,2.586357,3.592681,3.313225,3.201797,2.609914,2.860877
58,3.76935,3.706802,3.199146,1.937380,4.773407,2.125042,2.718904,2.636123,2.592616,2.647272,2.810628
59,3.14800,4.828891,2.736096,2.244158,5.848134,2.309203,2.990713,2.931414,2.171619,2.742558,3.189661


In [ ]:
inputs = Input(shape=(timesteps, 1))
x = LSTM(61)(inputs)
x = Dense(32)(x)
x = Dense(16)(x)
x = Dense(8)(x)
encoded = Dense(1)(x)

# decoded = Dense(16)(encoded)
# decoded = Dense(32)(decoded)
# decoded = Dense(61)(decoded)
# decoded = Reshape((61, 1))(decoded)
# decoded = LSTM(1, return_sequences=True)(decoded)
decoded = Dense(timesteps)(encoded)
decoded = Reshape((timesteps, 1))(decoded)
decoded = LSTM(61, return_sequences=True)(decoded)
decoded = Dense(32)(decoded)
decoded = Dense(16)(decoded)
decoded = Dense(1)(decoded)

latent_space_model = Model(inputs, encoded)
autoencoder = Model(inputs, decoded)
autoencoder.compile(optimizer='adam', loss='mae')


In [ ]:
autoencoder.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 61, 1)]           0         
                                                                 
 lstm_8 (LSTM)               (None, 61)                15372     
                                                                 
 dense_24 (Dense)            (None, 32)                1984      
                                                                 
 dense_25 (Dense)            (None, 16)                528       
                                                                 
 dense_26 (Dense)            (None, 8)                 136       
                                                                 
 dense_27 (Dense)            (None, 1)                 9         
                                                                 
 dense_28 (Dense)            (None, 61)                122 

In [ ]:
for i in range(n_features):
    print(f'Training on column: {i+1}/{n_features}')
    x_train = data.iloc[:, i].values.reshape(-1, timesteps, 1)

    autoencoder.fit(x_train, x_train, epochs=20, batch_size=4,verbose=0)

# Save model weights
autoencoder.save_weights('autoencoder_weights.h5')

Training on column: 1/11
Training on column: 2/11
Training on column: 3/11
Training on column: 4/11
Training on column: 5/11
Training on column: 6/11
Training on column: 7/11
Training on column: 8/11
Training on column: 9/11
Training on column: 10/11
Training on column: 11/11


In [ ]:
def predict_sequence_and_latent_space(input_sequence):
    input_sequence = np.array(input_sequence)
    input_sequence = input_sequence.reshape(1, -1, 1)
    print(input_sequence.shape)
    predicted_sequence = autoencoder.predict(input_sequence)
    latent_space_output = latent_space_model.predict(input_sequence)
    return predicted_sequence.reshape(-1), latent_space_output[0][0]

column_to_predict = 0
input_sequence = data.iloc[:, column_to_predict].values.tolist()
predictions, latent_space_output = predict_sequence_and_latent_space(input_sequence)

print("Original Sequence:")
print(input_sequence)

print("Predicted Sequence:")
print(predictions)

print("Latent Space Output:")
print(latent_space_output)

(1, 61, 1)
1/1 [==============================] - 0s 30ms/step
Original Sequence:
[3.06397, 3.83101, 4.05691, 3.52221, 3.98526, 4.01302, 3.80077, 3.88568, 3.309, 3.35152, 3.14882, 2.93705, 3.55888, 2.90615, 3.69932, 2.49518, 2.62947, 3.53079, 3.33833, 3.17512, 2.53417, 3.37458, 3.84312, 2.82021, 2.95482, 2.70857, 2.72143, 3.12778, 3.3321, 3.99528, 2.52391, 2.26012, 2.56221, 2.4485, 3.66982, 3.84158, 3.11703, 2.68244, 2.82298, 5.40269, 2.94102, 2.98804, 3.43231, 2.86975, 4.03197, 5.01041, 3.4988, 4.84479, 3.96105, 5.31183, 2.76793, 5.74459, 2.65164, 2.77291, 2.92063, 3.03805, 2.77059, 4.50932, 3.76935, 3.148, 2.96375]
Predicted Sequence:
[2.5155427 2.9878566 3.206471  3.3914688 3.473533  3.3320024 3.2741933
 3.3666205 3.2258263 3.0991611 3.258601  3.3522122 3.3735065 3.3610477
 3.2143996 3.1663728 3.0227547 3.1466358 3.2277038 3.3144834 3.158132
 3.0682414 2.9566772 2.9977896 3.0852711 2.9968112 2.9964683 3.1808622
 3.1391892 3.1097703 3.0678701 3.194622  3.298168  3.408769  3.2818847
 

In [ ]:
import pandas as pd

# Convert the predicted sequence back to a DataFrame
predictions_df = pd.DataFrame(predictions, columns=['Predicted Sequence'])

# Get the original DataFrame
original_df = pd.DataFrame(input_sequence, columns = ['Original Sequence'])

# Get the column to predict (column_to_predict) and the corresponding column name
# column_name = original_df.columns[column_to_predict]

# Append the predicted sequence to the original DataFrame
original_df['Predicted Sequence'] = predictions_df

# Save the DataFrame to a CSV file
original_df.to_csv('predicted_sequences.csv', index=False)

print("Original Sequence and Predicted Sequence saved to 'predicted_sequences.csv'")


Original Sequence and Predicted Sequence saved to 'predicted_sequences.csv'


In [ ]:
num_columns = 11  # Total number of columns in your data

def predict_and_get_latent_output(input_sequence, column_idx):
    input_sequence = np.array(input_sequence)
    input_sequence = input_sequence.reshape(1, -1, 1)
    predicted_sequence = autoencoder.predict(input_sequence)
    latent_space_output = latent_space_model.predict(input_sequence)
    return predicted_sequence.reshape(-1), latent_space_output.reshape(-1)

# Create an array to store the latent_space_op values for each column
latent_space_op_array = []

# Loop over each column
for column_idx in range(num_columns):
    input_sequence = data.iloc[:, column_idx].values.tolist()
    predictions, latent_space_output = predict_and_get_latent_output(input_sequence, column_idx)
    latent_space_op_array.append(latent_space_output)

print(latent_space_op_array)

1/1 [==============================] - 0s 47ms/step


In [ ]:
latent_space_op_array

[array([-121.90466], dtype=float32),
 array([-121.9372], dtype=float32),
 array([-121.953125], dtype=float32),
 array([-121.80627], dtype=float32),
 array([-121.94705], dtype=float32),
 array([-121.81171], dtype=float32),
 array([-122.00707], dtype=float32),
 array([-121.90172], dtype=float32),
 array([-121.91606], dtype=float32),
 array([-121.862305], dtype=float32),
 array([-121.874535], dtype=float32)]

Data saved to output_data.csv


In [ ]:
data = [['Latent Space Output for each column:']]
for item in latent_space_op_array:
    data.append([item[0]])

# Write data to CSV file
csv_filename = 'latent_space_output.csv'
with open(csv_filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(data)
